# Chapter 30: Graphs

In [1]:
from pyspark.sql import SparkSession
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.5.0-spark2.1-s_2.11 pyspark-shell'
spark = SparkSession.builder.master("local").appName("Chapter30").getOrCreate()

In [2]:
bikeStations = spark.read.format("csv").option("header", "True").load("../data/bike-data/201508_station_data.csv")
bikeStations.show(5)

+----------+--------------------+---------+-----------+---------+--------+------------+
|station_id|                name|      lat|       long|dockcount|landmark|installation|
+----------+--------------------+---------+-----------+---------+--------+------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|San Jose|    8/6/2013|
|         3|San Jose Civic Ce...|37.330698|-121.888979|       15|San Jose|    8/5/2013|
|         4|Santa Clara at Al...|37.333988|-121.894902|       11|San Jose|    8/6/2013|
|         5|    Adobe on Almaden|37.331415|  -121.8932|       19|San Jose|    8/5/2013|
|         6|    San Pedro Square|37.336721|-121.894074|       15|San Jose|    8/7/2013|
+----------+--------------------+---------+-----------+---------+--------+------------+
only showing top 5 rows



In [3]:
tripData = spark.read.format("csv").option("header", "True").load("../data/bike-data/201508_trip_data.csv")
tripData.show(5)

+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|       Start Station|Start Terminal|       End Date|         End Station|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913460|     765|8/31/2015 23:26|Harry Bridges Pla...|            50|8/31/2015 23:39|San Francisco Cal...|          70|   288|     Subscriber|    2139|
| 913459|    1036|8/31/2015 23:11|San Antonio Shopp...|            31|8/31/2015 23:28|Mountain View Cit...|          27|    35|     Subscriber|   95032|
| 913455|     307|8/31/2015 23:13|      Post at Kearny|            47|8/31/2015 23:18|   2nd at South Park|          64|   468|     Subscriber|   94107|
| 913454|     409|8/31/2015 23:10|  San Jose City Hall|            10|8/31/2015 23

In [4]:
# build the directed graph
stationVertices = bikeStations.withColumnRenamed("name", "id").distinct()
tripEdges = tripData.withColumnRenamed("Start Station", "src").withColumnRenamed("End Station", "dst")

from graphframes import GraphFrame
stationGraph = GraphFrame(stationVertices, tripEdges)
stationGraph.cache()

print("Number of stations (vertices) = {}".format(stationGraph.vertices.count()))
print("Number of trips in graph (edges) = {}".format(stationGraph.edges.count()))
print("Number of trips in original data = {}".format(tripData.count()))

Number of stations (vertices) = 70
Number of trips in graph (edges) = 354152
Number of trips in original data = 354152


## Querying the graph

The object `graph.edges` and `graph.vertices` are spark dataframes and can be queried as such, obviously.

from pyspark.sql.functions import desc

stationGraph.edges.groupBy(["src", "dst"]).count().orderBy(desc("count")).show(10)

In [5]:
# number of trips in an out of station
from pyspark.sql.functions import desc

stationGraph.edges.where("src = 'Townsend at 7th' OR dst = 'Townsend at 7th'")\
                  .groupby("src", "dst").count().orderBy(desc("count")).show(10)

+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|San Francisco Cal...|     Townsend at 7th| 3748|
|     Townsend at 7th|San Francisco Cal...| 2734|
|     Townsend at 7th|San Francisco Cal...| 2192|
|     Townsend at 7th|Civic Center BART...| 1844|
|Civic Center BART...|     Townsend at 7th| 1765|
|San Francisco Cal...|     Townsend at 7th| 1198|
|Temporary Transba...|     Townsend at 7th|  834|
|     Townsend at 7th|Harry Bridges Pla...|  827|
|   Steuart at Market|     Townsend at 7th|  746|
|     Townsend at 7th|Temporary Transba...|  740|
+--------------------+--------------------+-----+
only showing top 10 rows



## Subgraphs

Given that under the hook a graph it is just a combination of two dataframes (vertices, edges) we can create subgraphs by querying the bigger graph.

In [6]:
# edges must be filtered, but not vertices
subEdges = stationGraph.edges.where("src = 'Townsend at 7th' OR dst = 'Townsend at 7th'")
subGraph = GraphFrame(stationGraph.vertices, subEdges)

## Motif finding

Motifs are structural patterns in the graph, the Neo4j syntax is used to specify them.

In [7]:
# (a) represents vertix a
# [ab] represent edge ab

# next motif is for circular itineraries through 3 stations (a, b, c, a)
motifs = stationGraph.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[ca]->(a)")\

# we want now to query for the fastest bike trip through 3 stations
# we must parse timestamps 
# ensure that it is the same bike all along
# ensure that all stations are different
# ensure that the trip order is respected through time
from pyspark.sql.functions import expr

motifs.selectExpr("*", 
                  "to_timestamp(ab.`Start Date`, 'MM/dd/yyyy/HH:mm') as abStart", 
                  "to_timestamp(bc.`Start Date`, 'MM/dd/yyyy/HH:mm') as bcStart", 
                  "to_timestamp(ca.`Start Date`, 'MM/dd/yyyy/HH:mm') as caStart")\
      .where("ca.`Bike #` = bc.`Bike #`").where("ab.`Bike #` = bc.`Bike #`")\
      .where("a.id != b.id").where("b.id != c.id")\
      .where("abStart < bcStart").where("bcStart < caStart")\
      .orderBy(expr("cast(caStart as long) - cast(abStart as long)"))\
      .selectExpr("a.id", "b.id", "c.id", "ab.`Start Date`", "ca.`End Date`")\
      .limit(1).show(1, False)

+---+---+---+----------+--------+
|id |id |id |Start Date|End Date|
+---+---+---+----------+--------+
+---+---+---+----------+--------+



## Pagerank


In [8]:
ranks = stationGraph.pageRank(resetProbability=0.15, maxIter=10)
ranks.vertices.orderBy(desc("pageRank")).select("id", "pageRank").show(10)

+--------------------+------------------+
|                  id|          pageRank|
+--------------------+------------------+
|San Jose Diridon ...| 4.051504835989922|
|San Francisco Cal...|3.3511832964279518|
|Mountain View Cal...|  2.51439077101569|
|Redwood City Calt...| 2.326308771371272|
|San Francisco Cal...|2.2311442913697364|
|Harry Bridges Pla...|1.8251120118883524|
|     2nd at Townsend|1.5821217785041168|
|Santa Clara at Al...|1.5730074084908332|
|     Townsend at 7th| 1.568456580534273|
|Embarcadero at Sa...|1.5414242087749768|
+--------------------+------------------+
only showing top 10 rows



## Directed graphs: in-degree and out-degree

In [10]:
# stations with highest incoming trips
stationGraph.inDegrees.orderBy(desc("inDegree")).show(5, False)

# stationsn with highest outgoing trips
stationGraph.outDegrees.orderBy(desc("outDegree")).show(5, False)

+----------------------------------------+--------+
|id                                      |inDegree|
+----------------------------------------+--------+
|San Francisco Caltrain (Townsend at 4th)|34810   |
|San Francisco Caltrain 2 (330 Townsend) |22523   |
|Harry Bridges Plaza (Ferry Building)    |17810   |
|2nd at Townsend                         |15463   |
|Townsend at 7th                         |15422   |
+----------------------------------------+--------+
only showing top 5 rows

+---------------------------------------------+---------+
|id                                           |outDegree|
+---------------------------------------------+---------+
|San Francisco Caltrain (Townsend at 4th)     |26304    |
|San Francisco Caltrain 2 (330 Townsend)      |21758    |
|Harry Bridges Plaza (Ferry Building)         |17255    |
|Temporary Transbay Terminal (Howard at Beale)|14436    |
|Embarcadero at Sansome                       |14158    |
+------------------------------------------

## Breadth-First-Search

Will search on how to connect two nodes (or set of nodes) based on the edges of the graph.

In [12]:
stationGraph.bfs(fromExpr="id = 'Townsend at 7th'", toExpr="id = 'Spear at Folsom'", maxPathLength=2).show(10)

+--------------------+--------------------+--------------------+
|                from|                  e0|                  to|
+--------------------+--------------------+--------------------+
|[65, Townsend at ...|[913371, 663, 8/3...|[49, Spear at Fol...|
|[65, Townsend at ...|[913265, 658, 8/3...|[49, Spear at Fol...|
|[65, Townsend at ...|[911919, 722, 8/3...|[49, Spear at Fol...|
|[65, Townsend at ...|[910777, 704, 8/2...|[49, Spear at Fol...|
|[65, Townsend at ...|[908994, 1115, 8/...|[49, Spear at Fol...|
|[65, Townsend at ...|[906912, 892, 8/2...|[49, Spear at Fol...|
|[65, Townsend at ...|[905201, 980, 8/2...|[49, Spear at Fol...|
|[65, Townsend at ...|[904010, 969, 8/2...|[49, Spear at Fol...|
|[65, Townsend at ...|[903375, 850, 8/2...|[49, Spear at Fol...|
|[65, Townsend at ...|[899944, 910, 8/2...|[49, Spear at Fol...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



## Connected components

It basically finds islands (subgraphs that are not connected to the main graph).
It runs on undirected graphs.

For demonstration purposes it will be run on this dataset.

In [13]:
# needed to run connected components
spark.sparkContext.setCheckpointDir("/tmp/checkpoints")

In [15]:
# undersample the graph to make it runnable
minGraph = GraphFrame(stationGraph.vertices, stationGraph.edges.sample(0.1, False))
cc = minGraph.connectedComponents()  # result depends on the sample taken

In [16]:
cc.where("component != 0").show()

+----------+--------------------+---------+-----------+---------+-------------+------------+------------+
|station_id|                  id|      lat|       long|dockcount|     landmark|installation|   component|
+----------+--------------------+---------+-----------+---------+-------------+------------+------------+
|        33|Rengstorff Avenue...|37.400241|-122.099076|       15|Mountain View|   8/16/2013|  8589934592|
|        25|Stanford in Redwo...| 37.48537|-122.203288|       15| Redwood City|   8/12/2013|  8589934592|
|        84|         Ryland Park|37.342725|-121.895617|       15|     San Jose|    4/9/2014|  8589934592|
|        12|SJSU 4th at San C...|37.332808|-121.883891|       19|     San Jose|    8/7/2013|  8589934592|
|        16|SJSU - San Salvad...|37.333955|-121.877349|       15|     San Jose|    8/7/2013|  8589934592|
|        36|California Ave Ca...|37.429082|-122.142805|       15|    Palo Alto|   8/14/2013|  8589934592|
|        80|Santa Clara Count...|37.352601|-12

## Strongly connected graphs

In [17]:
scc = minGraph.stronglyConnectedComponents(maxIter=3)
scc.groupBy("component").count().show()

+------------+-----+
|   component|count|
+------------+-----+
|128849018880|   16|
|  8589934592|   19|
|           0|   33|
| 17179869184|    1|
|317827579904|    1|
+------------+-----+

